<h3>Create a Gunicorn Systemd Service File:</h3>


```bash
sudo nano /etc/systemd/system/gunicorn.service
```
<pre><code>
[Unit]
Description=gunicorn daemon for myproject
After=network.target

[Service]
User=yourusername
Group=yourusername
WorkingDirectory=/path/to/your/project
ExecStart=/path/to/your/project/venv/bin/gunicorn --workers 3 --bind unix:/path/to/your/project/myproject.sock myproject.wsgi:application

[Install]
WantedBy=multi-user.target
</code></pre>


<p>then start the service</p>
    
```bash
sudo systemctl start myproject
sudo systemctl enable myproject
```



<h3>install nginx</h3>

```bash
sudo apt install nginx
```

```bash
sudo nano /etc/nginx/sites-available/myproject
```
<pre><code>
server {
    listen 80;
    server_name your_domain_or_IP;

    location = /favicon.ico { access_log off; log_not_found off; }
    location /static/ {
        root /path/to/your/project;
    }
    
    location /media/ {
        root /path/to/your/project;
    }

    location / {
        include proxy_params;
        proxy_pass http://unix:/path/to/your/project/myproject.sock;
    }
}

</code></pre>

<h3>Verify Nginx User:</h3>

```bash
sudo usermod -a -G kshyst www-data
```

<h3>Enable the New Nginx Configuration:</h3>

```bash
sudo ln -s /etc/nginx/sites-available/myproject /etc/nginx/sites-enabled
```

<h3>Test and Restart Nginx:</h3>

```bash
sudo nginx -t
sudo systemctl daemon-reload
sudo systemctl restart myproject
sudo systemctl restart nginx
```

<h3>Firewall</h3>

```bash
sudo ufw allow 'Nginx Full'
```

<h3>Logging errors of nginx</h3>

```bash
sudo tail -f /var/log/nginx/error.log
```

<h3>Creating name servers</h3>

```bash
sudo apt update
sudo apt install bind9 bind9utils bind9-doc
```

<p>add zone configurations to your domain</p>
```bash
sudo nano /etc/bind/named.conf.local
```

<pre><code>
zone "kshyst.ir" {
    type master;
    file "/etc/bind/zones/db.kshyst.ir";
};

zone "0.168.192.in-addr.arpa" {
    type master;
    file "/etc/bind/zones/db.192.168.0";
};

</code></pre>

<p>Configure BIND</p>
```bash
sudo nano /etc/bind/db.kshyst.ir
```

<pre><code>
$TTL 86400
@   IN  SOA  ns1.kshyst.ir. admin.kshyst.ir. (
            2024072501 ; Serial
            3600       ; Refresh
            1800       ; Retry
            1209600    ; Expire
            86400 )    ; Minimum TTL
;
@   IN  NS  ns1.kshyst.ir.
@   IN  NS  ns2.kshyst.ir.
;
@   IN  A   <Your Server's IP Address>
ns1 IN  A   <Your First DNS Server's IP Address>
ns2 IN  A   <Your Second DNS Server's IP Address>

</code></pre>

<p>remove ns2 incase of 1 ip address</p>

```bash
sudo nano /etc/bind/zones/db.192.168.0
```

<pre><code>
$TTL 86400
@   IN  SOA  ns1.kshyst.ir. admin.kshyst.ir. (
            2024072501 ; Serial
            3600       ; Refresh
            1800       ; Retry
            1209600    ; Expire
            86400 )    ; Minimum TTL
;
@   IN  NS  ns1.kshyst.ir.
@   IN  NS  ns2.kshyst.ir.
;
1   IN  PTR kshyst.ir.

</code></pre>

<h3>Setup and test</h3>

```bash
sudo named-checkconf
sudo named-checkzone kshyst.ir /etc/bind/zones/db.kshyst.ir

sudo systemctl restart bind9
```

<p>Verify Setup</p>
```bash
dig @localhost kshyst.ir
```